In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
import os
from typing import Union, Tuple

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

In [37]:
df = pd.read_csv("data/EMEWS_cleaned_with_nan.csv")
df.head()

,day,date,total_number_of_patients,total_number_of_emews,zone_a_mwr_patients,zone_a_mwr_cat_3,zone_a_mwr_cat_4,zone_a_mwr_sets_of_emews,zone_a_mwr_deescalations,zone_a_mwr_escalations,...,zone_a__cat_3,zone_a__sets_of_emews,zone_a__deescalations,zone_a__escalations,zone_b/c_patients,zone_b/c_cat_2,zone_b/c_cat_3,zone_b/c_sets_of_emews,zone_b/c_deescalations,zone_b/c_escalations
0,thursday,2024-01-04 00:00:00,100.0,302.0,64.0,53.0,11.0,192.0,10.0,6.0,...,9.0,50.0,9.0,1.0,15.0,NaN,NaN,60.0,1.0,5.0
1,friday,2024-01-05 00:00:00,112.0,220.0,59.0,59.0,0.0,90.0,10.0,0.0,...,12.0,60.0,10.0,0.0,27.0,12.0,15.0,70.0,0.0,5.0
2,saturday,2024-01-06 00:00:00,69.0,175.0,36.0,36.0,0.0,58.0,3.0,2.0,...,0.0,67.0,5.0,2.0,15.0,NaN,NaN,50.0,0.0,5.0
3,sunday,2024-01-07 00:00:00,74.0,206.0,30.0,27.0,3.0,41.0,2.0,11.0,...,6.0,61.0,7.0,3.0,16.0,NaN,NaN,50.0,0.0,5.0
4,monday,2024-01-08 00:00:00,87.0,267.0,41.0,38.0,3.0,116.0,10.0,1.0,...,5.0,73.0,8.0,1.0,23.0,10.0,13.0,78.0,0.0,4.0


In [38]:
date_req = pd.to_datetime('2024-01-17')
df['date'] = pd.to_datetime(df['date'])

new_row = pd.DataFrame([{col: np.nan for col in df.columns}])
new_row['date'] = date_req
new_row['day'] = date_req.day_name().lower()


df = pd.concat([new_row, df], ignore_index=True)
df.sort_values('date', inplace=True)
df.reset_index(drop=True, inplace=True)

In [39]:
split_idx = int(len(df) * 0.9)
train_data = df.iloc[:split_idx]
test_data = df.iloc[split_idx:]

# Then split X and y
X_train = train_data.drop(columns=['total_number_of_patients', 'total_number_of_emews'])
y_train = train_data[['total_number_of_patients', 'total_number_of_emews']]

X_test = test_data.drop(columns=['total_number_of_patients', 'total_number_of_emews'])
y_test = test_data[['total_number_of_patients', 'total_number_of_emews']]

# X_train = train_df.drop(columns=['total_number_of_patients'])
# y_train = train_df[['total_number_of_patients']]

# X_test = test_df.drop(columns=['total_number_of_patients'])
# y_test = test_df[['total_number_of_patients']]

In [40]:
def apply_imputation(
        X_train: pd.DataFrame, 
        X_test: pd.DataFrame,
        imputer: Union[KNNImputer, IterativeImputer, SimpleImputer]
        ) -> Tuple[pd.DataFrame, pd.DataFrame]:

    num_cols = X_train.select_dtypes(include='number').columns
    non_num_cols = X_train.columns.difference(num_cols).to_list()

    X_train_num = X_train[num_cols]
    X_test_num = X_test[num_cols]

    needs_scaling = type(imputer) in (KNNImputer, IterativeImputer)

    if needs_scaling:
        scaler = StandardScaler()
        X_train_num = scaler.fit_transform(X_train_num)
        X_test_num = scaler.transform(X_test_num)

    X_train_imputed = imputer.fit_transform(X_train_num)
    X_test_imputed = imputer.transform(X_test_num)

    if needs_scaling:
        X_train_imputed = scaler.inverse_transform(X_train_imputed)
        X_test_imputed = scaler.inverse_transform(X_test_imputed)

    X_train_imputed = np.round(X_train_imputed).astype(int)
    X_test_imputed = np.round(X_test_imputed).astype(int)

    # Replace only numeric columns
    X_train_copy = X_train.copy()
    X_train_copy[num_cols] = X_train_imputed
    X_train_copy[non_num_cols] = X_train[non_num_cols]

    X_test_copy = X_test.copy()
    X_test_copy[num_cols] = X_test_imputed
    X_test_copy[non_num_cols] = X_test[non_num_cols]

    return X_train_copy, X_test_copy

In [41]:
imputation_strategies = {
    'mean': SimpleImputer(strategy='mean'),
    'median': SimpleImputer(strategy='median'),
    'mode': SimpleImputer(strategy='most_frequent'),
    'mice': IterativeImputer(random_state=42),
    'mice_hgb': IterativeImputer(estimator=HistGradientBoostingRegressor(random_state=42), random_state=42),
    'mice_lr': IterativeImputer(estimator=LinearRegression(), random_state=42),
    'mice_rf': IterativeImputer(estimator=RandomForestRegressor(random_state=42), random_state=42),
    'mice_svr': IterativeImputer(estimator=SVR(), random_state=42)
}

for i in range(3, 6):
    key = f'knn_{i}'
    imputation_strategies[key] = KNNImputer(n_neighbors=i)
    key = f'knn_{i}_distance'
    imputation_strategies[key] = KNNImputer(n_neighbors=i, weights='distance')
    key = f'mice_knn_{i}'
    imputation_strategies[key] = IterativeImputer(estimator=KNeighborsRegressor(n_neighbors=i), random_state=42)
    key = f'mice_knn_{i}_distance'
    imputation_strategies[key] = IterativeImputer(estimator=KNeighborsRegressor(n_neighbors=i, weights='distance'), random_state=42)
    

datasets = {}

for name, imputer in imputation_strategies.items():
    X_train_imp, X_test_imp = apply_imputation(X_train, X_test, imputer)

    train_df = pd.concat([X_train_imp, y_train], axis=1)
    test_df = pd.concat([X_test_imp, y_test], axis=1)

    train_df['date'] = pd.to_datetime(train_df['date'])
    test_df['date'] = pd.to_datetime(test_df['date'])

    # manual checks...
    patient_count_cols = ['zone_a_mwr_patients', 'zone_a__patients', 'zone_b/c_patients']
    emews_count_cols = ['zone_a_mwr_sets_of_emews', 'zone_a__sets_of_emews', 'zone_b/c_sets_of_emews']
    
    train_df['total_number_of_patients'] = train_df['total_number_of_patients'].fillna(train_df[patient_count_cols].sum(axis=1))
    test_df['total_number_of_patients'] = test_df['total_number_of_patients'].fillna(test_df[patient_count_cols].sum(axis=1))

    train_df['total_number_of_emews'] = train_df['total_number_of_emews'].fillna(train_df[emews_count_cols].sum(axis=1))
    test_df['total_number_of_emews'] = test_df['total_number_of_emews'].fillna(test_df[emews_count_cols].sum(axis=1))

    # train_df['corrected_total_number_of_patients'] = train_df[patient_count_cols].sum(axis=1)
    # test_df['corrected_total_number_of_patients'] = test_df[patient_count_cols].sum(axis=1)

    datasets[name] = (train_df, test_df)

In [42]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def r2_rounded(y_true, y_pred):
    """Calculates R2 score after rounding predictions to the nearest whole number."""
    return r2_score(y_true, np.round(y_pred))

def rmse_rounded(y_true, y_pred):
    """Calculates RMSE after rounding predictions to the nearest whole number."""
    return np.sqrt(mean_squared_error(y_true, np.round(y_pred)))

def r2_ceil(y_true, y_pred):
    """Calculates R2 score after ceiling predictions to the nearest whole number."""
    return r2_score(y_true, np.ceil(y_pred))

def rmse_ceil(y_true, y_pred):
    """Calculates RMSE after ceiling predictions to the nearest whole number."""
    return np.sqrt(mean_squared_error(y_true, np.ceil(y_pred)))

def mae_rounded(y_true, y_pred):
    """Calculates MAE after rounding predictions to the nearest whole number."""
    return mean_absolute_error(y_true, np.round(y_pred))

def mae_ceil(y_true, y_pred):
    """Calculates MAE after ceiling predictions to the nearest whole number."""
    return mean_absolute_error(y_true, np.ceil(y_pred))

In [43]:
from pycaret.regression import *
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit

results = []

for name, (train_df, test_df) in datasets.items():
    print(f"\n🔍 Processing dataset: {name}")
    # train_df = train_df.copy().drop(columns=['corrected_total_number_of_patients'])
    # test_df = test_df.copy().drop(columns=['corrected_total_number_of_patients'])

    setup(
        data=train_df,
        target='total_number_of_patients',
        session_id=42,
        verbose=False,
        test_data=test_df,  # Use predefined test set,
        fold_strategy='timeseries',
        data_split_shuffle=False,
        fold_shuffle=False
    )

    best_model = compare_models(exclude=['lightgbm', 'par', 'dummy', 'lar'], sort='R2')
    leaderboard = pull()
    top_model_results = leaderboard.iloc[0]

    preds = predict_model(best_model, data=test_df)
    
    y_true = test_df['total_number_of_patients'].values
    y_pred = preds['prediction_label'].values

    # Standard metrics
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)

    # Custom metrics
    r2_r = r2_rounded(y_true, y_pred)
    rmse_r = rmse_rounded(y_true, y_pred)
    r2_c = r2_ceil(y_true, y_pred)
    rmse_c = rmse_ceil(y_true, y_pred)
    mae_r = mae_rounded(y_true, y_pred)
    mae_c = mae_ceil(y_true, y_pred)

    results.append({
        'dataset': name,
        'model': top_model_results['Model'],
        'R2_train': top_model_results['R2'],
        'R2_test': r2,
        'RMSE_train': top_model_results['RMSE'],
        'RMSE_test': rmse,
        'MAE_train': top_model_results['MAE'],
        'MAE_test': mae,
        'R2_rounded': r2_r,
        'RMSE_rounded': rmse_r,
        'MAE_rounded': mae_r,
        'R2_ceil': r2_c,
        'RMSE_ceil': rmse_c,
        'MAE_ceil': mae_c
    })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='R2_test', ascending=False)
print("\n📊 Final Results:")
results_df



🔍 Processing dataset: mean


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,2.8704,35.2056,5.5136,0.9120,0.3106,0.0955,0.0190
lasso,Lasso Regression,4.4972,44.1738,6.2896,0.8892,0.3720,0.1604,0.2930
llar,Lasso Least Angle Regression,4.4972,44.1731,6.2895,0.8892,0.3719,0.1604,0.0160
en,Elastic Net,4.5194,44.9587,6.3342,0.8877,0.3720,0.1589,0.0150
br,Bayesian Ridge,4.4659,46.5528,6.4912,0.8823,0.3799,0.1551,0.0160
ridge,Ridge Regression,4.8270,53.0170,6.9475,0.8675,0.3777,0.1528,0.0150
lr,Linear Regression,4.8881,54.4063,7.0154,0.8650,0.3770,0.1537,0.6030
catboost,CatBoost Regressor,4.4126,57.9278,6.8521,0.8617,0.3260,0.1312,0.4740
et,Extra Trees Regressor,4.5360,60.2922,7.1393,0.8581,0.3201,0.1341,0.0430
gbr,Gradient Boosting Regressor,5.0050,63.3691,7.3965,0.8386,0.3218,0.1580,0.0270


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Huber Regressor,3.3223,28.8363,5.3699,0.9309,0.1128,0.0547



🔍 Processing dataset: median


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,2.8616,36.7857,5.5823,0.9095,0.3190,0.0941,0.0190
lasso,Lasso Regression,4.4388,44.6495,6.3034,0.8887,0.3671,0.1570,0.0170
llar,Lasso Least Angle Regression,4.4386,44.6472,6.3032,0.8887,0.3671,0.1570,0.0160
en,Elastic Net,4.4640,45.3953,6.3462,0.8875,0.3679,0.1550,0.0150
br,Bayesian Ridge,4.3930,46.8690,6.4900,0.8829,0.3765,0.1501,0.0250
ridge,Ridge Regression,4.7103,52.2162,6.8538,0.8715,0.3753,0.1440,0.0150
lr,Linear Regression,4.7729,53.7288,6.9302,0.8687,0.3748,0.1450,0.3800
catboost,CatBoost Regressor,4.4935,58.6576,6.9145,0.8620,0.3284,0.1328,13.5230
et,Extra Trees Regressor,4.8278,66.1164,7.4615,0.8453,0.3297,0.1436,0.0530
gbr,Gradient Boosting Regressor,5.0960,65.9519,7.6217,0.8303,0.3362,0.1637,0.0310


...


🔍 Processing dataset: mice_knn_5_distance


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,4.1768,40.5104,6.0863,0.8905,0.3226,0.1436,0.0190
lasso,Lasso Regression,5.0223,47.8063,6.6460,0.8738,0.3684,0.1662,0.0160
llar,Lasso Least Angle Regression,5.0220,47.8024,6.6457,0.8738,0.3684,0.1662,0.0160
en,Elastic Net,5.0486,48.4546,6.6772,0.8735,0.3655,0.1645,0.0150
br,Bayesian Ridge,5.0541,49.6795,6.7632,0.8693,0.3673,0.1656,0.0160
ridge,Ridge Regression,5.5908,65.2551,7.4203,0.8449,0.3693,0.1689,0.0150
lr,Linear Regression,5.7523,73.2213,7.6652,0.8316,0.3681,0.1712,0.0170
catboost,CatBoost Regressor,5.8220,82.0012,8.4638,0.7971,0.3601,0.1777,0.3960
et,Extra Trees Regressor,5.8852,85.9053,8.7481,0.7881,0.3424,0.1889,0.0430
gbr,Gradient Boosting Regressor,6.2386,87.7008,8.8532,0.7776,0.3448,0.1952,0.0310


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Huber Regressor,4.4232,40.6881,6.3787,0.9037,0.1047,0.0765



📊 Final Results:


,dataset,model,R2_train,R2_test,RMSE_train,RMSE_test,MAE_train,MAE_test,R2_rounded,RMSE_rounded,MAE_rounded,R2_ceil,RMSE_ceil,MAE_ceil
0,mean,Huber Regressor,0.9120,0.930880,5.5136,5.369945,2.8704,3.322315,0.931326,5.352569,3.250000,0.927052,5.516641,3.233333
1,median,Huber Regressor,0.9095,0.922313,5.5823,5.720601,2.8616,3.280030,0.920395,5.790797,3.333333,0.919643,5.818075,3.250000
2,mode,Huber Regressor,0.9010,0.912652,6.0425,6.061448,2.9933,3.603343,0.914255,6.005553,3.566667,0.906370,6.275614,3.716667
5,mice_lr,Huber Regressor,0.9234,0.911111,5.1476,6.109827,3.1821,3.798831,0.911381,6.100546,3.750000,0.906619,6.262321,3.816667
19,mice_knn_5_distance,Huber Regressor,0.8905,0.903712,6.0863,6.378724,4.1768,4.423230,0.903329,6.391400,4.383333,0.901159,6.462714,4.433333
18,mice_knn_5,Huber Regressor,0.8910,0.902049,6.0195,6.423521,4.1172,4.305810,0.901562,6.439462,4.266667,0.897685,6.565059,4.400000
15,mice_knn_4_distance,Huber Regressor,0.8883,0.899324,6.0642,6.589455,4.0469,4.201934,0.900107,6.563790,4.150000,0.894620,6.741662,4.283333
17,knn_5_distance,Huber Regressor,0.8828,0.895837,6.2037,6.689791,4.3095,4.309459,0.895379,6.704476,4.283333,0.893091,6.777413,4.333333
14,mice_knn_4,Huber Regressor,0.8883,0.894467,6.1037,6.746551,4.1135,4.285714,0.893692,6.771263,4.283333,0.890910,6.859300,4.350000
3,mice,Huber Regressor,0.9261,0.889505,4.9910,6.903329,3.1378,3.989016,0.888128,6.946222,4.050000,0.885423,7.029699,3.950000


In [44]:
results_df.sort_values(by='R2_rounded', ascending=False)

,dataset,model,R2_train,R2_test,RMSE_train,RMSE_test,MAE_train,MAE_test,R2_rounded,RMSE_rounded,MAE_rounded,R2_ceil,RMSE_ceil,MAE_ceil
0,mean,Huber Regressor,0.9120,0.930880,5.5136,5.369945,2.8704,3.322315,0.931326,5.352569,3.250000,0.927052,5.516641,3.233333
1,median,Huber Regressor,0.9095,0.922313,5.5823,5.720601,2.8616,3.280030,0.920395,5.790797,3.333333,0.919643,5.818075,3.250000
2,mode,Huber Regressor,0.9010,0.912652,6.0425,6.061448,2.9933,3.603343,0.914255,6.005553,3.566667,0.906370,6.275614,3.716667
5,mice_lr,Huber Regressor,0.9234,0.911111,5.1476,6.109827,3.1821,3.798831,0.911381,6.100546,3.750000,0.906619,6.262321,3.816667
19,mice_knn_5_distance,Huber Regressor,0.8905,0.903712,6.0863,6.378724,4.1768,4.423230,0.903329,6.391400,4.383333,0.901159,6.462714,4.433333
18,mice_knn_5,Huber Regressor,0.8910,0.902049,6.0195,6.423521,4.1172,4.305810,0.901562,6.439462,4.266667,0.897685,6.565059,4.400000
15,mice_knn_4_distance,Huber Regressor,0.8883,0.899324,6.0642,6.589455,4.0469,4.201934,0.900107,6.563790,4.150000,0.894620,6.741662,4.283333
17,knn_5_distance,Huber Regressor,0.8828,0.895837,6.2037,6.689791,4.3095,4.309459,0.895379,6.704476,4.283333,0.893091,6.777413,4.333333
14,mice_knn_4,Huber Regressor,0.8883,0.894467,6.1037,6.746551,4.1135,4.285714,0.893692,6.771263,4.283333,0.890910,6.859300,4.350000
3,mice,Huber Regressor,0.9261,0.889505,4.9910,6.903329,3.1378,3.989016,0.888128,6.946222,4.050000,0.885423,7.029699,3.950000


In [45]:
from pycaret.regression import setup, compare_models, pull
import pandas as pd

results = []

for name, (train_df, test_df) in datasets.items():
    print(f"\n🔍 Processing dataset: {name}")
    # train_df = train_df.copy().drop(columns=['corrected_total_number_of_patients'])
    # test_df = test_df.copy().drop(columns=['corrected_total_number_of_patients'])

    setup(
        data=train_df,
        target='total_number_of_emews',
        session_id=42,
        verbose=False,
        test_data=test_df,  # Use predefined test set,
        fold_strategy='timeseries',
        data_split_shuffle=False,
        fold_shuffle=False
    )

    best_model = compare_models(exclude=['lightgbm', 'par', 'dummy', 'lar'], sort='R2')
    leaderboard = pull()
    top_model_results = leaderboard.iloc[0]

    preds = predict_model(best_model, data=test_df)
    
    y_true = test_df['total_number_of_emews'].values
    y_pred = preds['prediction_label'].values

    # Standard metrics
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)

    # Custom metrics
    r2_r = r2_rounded(y_true, y_pred)
    rmse_r = rmse_rounded(y_true, y_pred)
    r2_c = r2_ceil(y_true, y_pred)
    rmse_c = rmse_ceil(y_true, y_pred)
    mae_r = mae_rounded(y_true, y_pred)
    mae_c = mae_ceil(y_true, y_pred)

    results.append({
        'dataset': name,
        'model': top_model_results['Model'],
        'R2_train': top_model_results['R2'],
        'R2_test': r2,
        'RMSE_train': top_model_results['RMSE'],
        'RMSE_test': rmse,
        'MAE_train': top_model_results['MAE'],
        'MAE_test': mae,
        'R2_rounded': r2_r,
        'RMSE_rounded': rmse_r,
        'MAE_rounded': mae_r,
        'R2_ceil': r2_c,
        'RMSE_ceil': rmse_c,
        'MAE_ceil': mae_c
    })

results_df_2 = pd.DataFrame(results)
results_df_2 = results_df_2.sort_values(by='R2_test', ascending=False)
print("\n📊 Final Results:")
results_df_2



🔍 Processing dataset: mean


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,7.0997,312.9918,16.1848,0.9267,0.4353,0.0557,0.0210
br,Bayesian Ridge,10.1029,336.3050,16.7265,0.9205,0.4533,0.0953,0.0160
en,Elastic Net,10.8984,356.3280,17.0704,0.9178,0.4084,0.1034,0.0170
lasso,Lasso Regression,11.8486,396.5323,18.1623,0.9101,0.4235,0.1093,0.0200
llar,Lasso Least Angle Regression,11.8492,396.5089,18.1620,0.9101,0.4236,0.1093,0.0200
catboost,CatBoost Regressor,13.7131,518.1401,20.8403,0.8885,0.4054,0.1143,0.7770
ridge,Ridge Regression,14.0581,521.6504,20.7265,0.8811,0.4902,0.1271,0.0170
gbr,Gradient Boosting Regressor,14.3899,550.0217,21.8917,0.8740,0.3961,0.1267,0.0300
lr,Linear Regression,14.8685,600.3832,21.6515,0.8683,0.4935,0.1341,0.0200
et,Extra Trees Regressor,14.4494,637.1850,22.4175,0.8676,0.2789,0.1128,0.0430


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Huber Regressor,9.1381,274.8858,16.5797,0.9457,0.0924,0.0472



🔍 Processing dataset: median


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,6.8021,291.6858,15.9132,0.9298,0.4655,0.0542,0.0200
br,Bayesian Ridge,9.4512,310.1910,16.0317,0.9270,0.4214,0.0893,0.0170
en,Elastic Net,10.2008,322.4664,16.2984,0.9254,0.3960,0.0969,0.0150
lasso,Lasso Regression,11.1603,366.6739,17.5064,0.9165,0.4121,0.1019,0.0160
llar,Lasso Least Angle Regression,11.1616,366.7303,17.5072,0.9165,0.4121,0.1020,0.0160
catboost,CatBoost Regressor,13.5691,497.4554,20.3825,0.8932,0.3884,0.1103,0.3590
ridge,Ridge Regression,12.9101,473.8482,19.8896,0.8881,0.4438,0.1161,0.0160
gbr,Gradient Boosting Regressor,14.4315,524.3023,21.2653,0.8818,0.3798,0.1215,0.0290
lr,Linear Regression,13.5859,536.3130,20.7041,0.8777,0.4558,0.1217,0.0200
et,Extra Trees Regressor,14.5643,629.5166,22.3636,0.8682,0.2776,0.1137,0.0430


...


🔍 Processing dataset: mice_knn_5_distance


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,10.8676,341.6913,17.0667,0.9172,0.3210,0.1036,0.0200
br,Bayesian Ridge,11.9136,375.8389,17.6038,0.9108,0.3998,0.1103,0.0160
en,Elastic Net,12.7554,396.9452,18.0617,0.9067,0.4282,0.1168,0.0150
lasso,Lasso Regression,14.2618,468.7436,19.6102,0.8928,0.4425,0.1268,0.0170
llar,Lasso Least Angle Regression,14.2615,468.7058,19.6098,0.8928,0.4425,0.1268,0.0170
ridge,Ridge Regression,16.0662,563.5427,21.4935,0.8701,0.5120,0.1430,0.0160
lr,Linear Regression,17.0160,661.5854,22.5658,0.8544,0.5206,0.1507,0.0300
catboost,CatBoost Regressor,16.0979,698.9238,24.1989,0.8467,0.4243,0.1399,0.7230
gbr,Gradient Boosting Regressor,16.7040,715.9570,24.5518,0.8401,0.3928,0.1435,0.0310
et,Extra Trees Regressor,16.1554,770.6927,24.9104,0.8382,0.3060,0.1314,0.0440


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Huber Regressor,13.5792,452.3763,21.2691,0.9098,0.1026,0.0707



📊 Final Results:


,dataset,model,R2_train,R2_test,RMSE_train,RMSE_test,MAE_train,MAE_test,R2_rounded,RMSE_rounded,MAE_rounded,R2_ceil,RMSE_ceil,MAE_ceil
1,median,Huber Regressor,0.9298,0.951102,15.9132,15.708514,6.8021,8.358915,0.950945,15.733722,8.283333,0.951790,15.597543,8.183333
0,mean,Huber Regressor,0.9267,0.945662,16.1848,16.579682,7.0997,9.138087,0.945557,16.595682,9.083333,0.946789,16.406808,9.083333
5,mice_lr,Huber Regressor,0.9375,0.945629,15.0253,16.366516,8.1957,9.764273,0.945618,16.368160,9.783333,0.946647,16.212649,9.750000
3,mice,Huber Regressor,0.9440,0.942365,14.1079,17.021567,7.9838,10.729172,0.942690,16.973509,10.700000,0.943463,16.858727,10.716667
2,mode,Huber Regressor,0.9407,0.928914,14.7901,18.819001,6.9152,10.919030,0.928918,18.818431,10.933333,0.930387,18.623014,10.650000
14,mice_knn_4,Huber Regressor,0.9107,0.921605,17.8051,19.823445,11.2550,11.998198,0.921735,19.806985,12.016667,0.922560,19.702369,11.916667
15,mice_knn_4_distance,Huber Regressor,0.9106,0.917191,17.6699,20.368510,11.0591,12.386157,0.917067,20.383817,12.400000,0.918414,20.217567,12.250000
17,knn_5_distance,Huber Regressor,0.9044,0.912094,18.1295,21.067819,11.4999,12.552424,0.911887,21.092653,12.566667,0.913544,20.893380,12.533333
8,knn_3,Huber Regressor,0.8926,0.910302,19.2868,21.318910,12.5906,14.048378,0.910732,21.267738,14.050000,0.910318,21.317051,14.083333
19,mice_knn_5_distance,Huber Regressor,0.9172,0.909770,17.0667,21.269141,10.8676,13.579168,0.909862,21.258332,13.516667,0.910447,21.189227,13.616667


In [46]:
results_df_2.sort_values(by='R2_rounded', ascending=False)

,dataset,model,R2_train,R2_test,RMSE_train,RMSE_test,MAE_train,MAE_test,R2_rounded,RMSE_rounded,MAE_rounded,R2_ceil,RMSE_ceil,MAE_ceil
1,median,Huber Regressor,0.9298,0.951102,15.9132,15.708514,6.8021,8.358915,0.950945,15.733722,8.283333,0.951790,15.597543,8.183333
5,mice_lr,Huber Regressor,0.9375,0.945629,15.0253,16.366516,8.1957,9.764273,0.945618,16.368160,9.783333,0.946647,16.212649,9.750000
0,mean,Huber Regressor,0.9267,0.945662,16.1848,16.579682,7.0997,9.138087,0.945557,16.595682,9.083333,0.946789,16.406808,9.083333
3,mice,Huber Regressor,0.9440,0.942365,14.1079,17.021567,7.9838,10.729172,0.942690,16.973509,10.700000,0.943463,16.858727,10.716667
2,mode,Huber Regressor,0.9407,0.928914,14.7901,18.819001,6.9152,10.919030,0.928918,18.818431,10.933333,0.930387,18.623014,10.650000
14,mice_knn_4,Huber Regressor,0.9107,0.921605,17.8051,19.823445,11.2550,11.998198,0.921735,19.806985,12.016667,0.922560,19.702369,11.916667
15,mice_knn_4_distance,Huber Regressor,0.9106,0.917191,17.6699,20.368510,11.0591,12.386157,0.917067,20.383817,12.400000,0.918414,20.217567,12.250000
17,knn_5_distance,Huber Regressor,0.9044,0.912094,18.1295,21.067819,11.4999,12.552424,0.911887,21.092653,12.566667,0.913544,20.893380,12.533333
8,knn_3,Huber Regressor,0.8926,0.910302,19.2868,21.318910,12.5906,14.048378,0.910732,21.267738,14.050000,0.910318,21.317051,14.083333
19,mice_knn_5_distance,Huber Regressor,0.9172,0.909770,17.0667,21.269141,10.8676,13.579168,0.909862,21.258332,13.516667,0.910447,21.189227,13.616667


In [47]:
# for name, (train_df, test_df) in datasets.items():
#     path = f'data/imputed/'
#     os.makedirs(path, exist_ok=True)

#     df = pd.concat([train_df, test_df], axis=0).sort_values(by='date').reset_index(drop=True)

#     df.to_csv(path+f'{name}_df.csv', index=False)

In [48]:
full_df = pd.concat([train_data, test_data], ignore_index=True)

X_full = full_df.drop(columns=['total_number_of_patients', 'total_number_of_emews'])
y_full = full_df[['total_number_of_patients', 'total_number_of_emews']]

patient_cols = ['zone_a_mwr_patients', 'zone_a__patients', 'zone_b/c_patients']
emews_cols = ['zone_a_mwr_sets_of_emews', 'zone_a__sets_of_emews', 'zone_b/c_sets_of_emews']

In [49]:
def apply_final_imputation(X: pd.DataFrame, imputer):
    num_cols = X.select_dtypes(include='number').columns

    X_num = X[num_cols]
    needs_scaling = isinstance(imputer, (KNNImputer, IterativeImputer))

    if needs_scaling:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_num)
        X_imputed = imputer.fit_transform(X_scaled)
        X_imputed = scaler.inverse_transform(X_imputed)
    else:
        X_imputed = imputer.fit_transform(X_num)

    X_imputed = np.round(X_imputed).astype(int)
    X_final = X.copy()
    X_final[num_cols] = X_imputed
    return X_final

In [50]:
imputed_datasets = {}

for name, imputer in imputation_strategies.items():
    print(f"🔧 Imputing with: {name}")

    X_imputed = apply_final_imputation(X_full.copy(), imputer)

    # Combine with target values again
    df_imputed = pd.concat([X_imputed, y_full.copy()], axis=1)

    # Fix totals if needed
    df_imputed['total_number_of_patients'] = df_imputed['total_number_of_patients'].fillna(
        df_imputed[patient_cols].sum(axis=1)
    )
    df_imputed['total_number_of_emews'] = df_imputed['total_number_of_emews'].fillna(
        df_imputed[emews_cols].sum(axis=1)
    )

    # Save to dictionary
    imputed_datasets[name] = df_imputed

🔧 Imputing with: mean
🔧 Imputing with: median
🔧 Imputing with: mode
🔧 Imputing with: mice
🔧 Imputing with: mice_hgb
🔧 Imputing with: mice_lr
🔧 Imputing with: mice_rf
🔧 Imputing with: mice_svr
🔧 Imputing with: knn_3
🔧 Imputing with: knn_3_distance
🔧 Imputing with: mice_knn_3
🔧 Imputing with: mice_knn_3_distance
🔧 Imputing with: knn_4
🔧 Imputing with: knn_4_distance
🔧 Imputing with: mice_knn_4
🔧 Imputing with: mice_knn_4_distance
🔧 Imputing with: knn_5
🔧 Imputing with: knn_5_distance
🔧 Imputing with: mice_knn_5
🔧 Imputing with: mice_knn_5_distance


In [51]:
for name, dataframe in imputed_datasets.items():
    path = f'data/imputed2/'
    os.makedirs(path, exist_ok=True)

    dataframe.to_csv(path+f'{name}_df.csv', index=False)